## opg_t03_base_0519_npd1
基于 opg_aftertraining0519.ipynb
本代码主要完成ninapro db1 同 subject 下，不同的模型已知类数量对结果的影响验证工作。

In [ ]:
# -*- coding:utf-8 -*-
import datetime
import os
import sys

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.io as scio
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import torch.optim as optim
import torchvision.transforms as transforms
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegressionCV
from sklearn import metrics
from torch.utils.data import DataLoader, Dataset
from torchsummary import summary
from torchvision import transforms
from torchviz import make_dot
# import hiddenlayer as h
from visdom import Visdom

# from sklearn.preprocessing import label_binarize
# from sklearn import preprocessing
# from sklearn import tree 

sys.path.append("..")
from utils.networks import *
from utils.reuse import *

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

## 预备操作
设置检查点、visdom 日志文件存储等日志性文件存储位置；
初始化 visdom,记得先在命令行输入 visdom 运行（python环境下）

In [ ]:
# 以下是检查点路径
# 请在当前环境下 CMD 输入python -m visdom.server 或 visdom 启动监视器
# 数据处理现在已移至 emgDataprocess.ipynb
# 现在model_Dir 作为所有文件的父目录，不再分设开导致文件难寻
model_Dir = "..//model//opg_testops_t03.2_220712a//s1//"
if not os.path.exists(model_Dir):
    os.makedirs(model_Dir)
# 这是正常定期检查点存储位置
ckpDir = model_Dir + "ckp//"
if not os.path.exists(ckpDir):
    os.makedirs(ckpDir)
# 这里是特挑最佳AUC的位置
ckpDir_auc = ckpDir + "auc//"
if not os.path.exists(ckpDir_auc):
    os.makedirs(ckpDir_auc)
# 这是 visdom 日志文件存储位置，留待备用
vislogDir = model_Dir + "vislog//"
if not os.path.exists(vislogDir):
    os.makedirs(vislogDir)

def get_current_time():
    return datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")


print(get_current_time())

timeForSave = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")

In [ ]:
# 以下是 visdom 监视窗口初始化，实现每次启用时重新加载，这里只写了 NameError 以防其他错误不能被发现
class visdom_account:
    def __init__(self):
        self.port = 8097
        self.server = "http://localhost"
        self.base_url = "/"
        self.username = "admin"
        self.passward = "1234"
        self.evns = "train_opg_t03_s1"


viz_acnt = visdom_account()


def viz_init():
    try:
        viz
    except NameError:
        viz = Visdom(
            env=viz_acnt.evns, log_to_filename=vislogDir + "vislog_" + timeForSave
        )
        print("visdom has started")
    else:
        viz.close()
        del viz
        print("last visdom session closed")
        viz = Visdom(
            env=viz_acnt.evns, log_to_filename=vislogDir + "vislog_" + timeForSave
        )
        print("visdom has restarted")
    return viz


viz = viz_init()

## GAN 模型训练

In [ ]:
# EMG需要转化成6维特征，用到刚刚训练的模型c_ep_200_acc9763_2022_02_26_23_18_44,
# 并重组为新数据集OpenganDatafea_smr_10cl_220227.npy，步骤见 emgDataprocess.ipynb
num_kc = 4
# opGAN部分开始
# GAN 模型搭建
# 潜在张量大小,32，4是因为 cnn feature 只有6个，不好比它大
latent_size = 4
# 输出通道数
n_channel = num_kc
# 生成网络隐藏层大小
n_g_feature = 64

gnet = nn.Sequential(
    nn.Flatten(),
    nn.Linear(latent_size, n_g_feature * 1),  # 用线性变换将输入映射到64维
    nn.BatchNorm1d(n_g_feature * 1),
    nn.ReLU(True),  # relu激活
    nn.Dropout2d(0.2),
    nn.Linear(n_g_feature * 1, n_g_feature * 2),  # 线性变换
    nn.BatchNorm1d(n_g_feature * 2),
    nn.ReLU(True),  # relu激活
    nn.Dropout2d(0.2),
    nn.Linear(n_g_feature * 2, n_channel),  # 线性变换
).to(device)
gnet.to("cpu")
# print(gnet)
summary(gnet, (1, 1, 4), batch_size=1, device="cpu")
gnet.to(device)
# 鉴别网络隐藏层大小
# 32,
n_d_feature = 64
dnet = nn.Sequential(
    nn.Flatten(),
    nn.Linear(n_channel, n_d_feature * 2),  # 输入特征数为784，输出为256
    nn.BatchNorm1d(n_d_feature * 2),
    nn.LeakyReLU(0.2),  # 进行非线性映射
    nn.Dropout2d(0.2),
    nn.Linear(n_d_feature * 2, n_d_feature),  # 进行一个线性映射
    nn.BatchNorm1d(n_d_feature),
    nn.LeakyReLU(0.2),
    nn.Dropout2d(0.2),
    nn.Linear(n_d_feature, 1),
    # nn.Sigmoid()  # 也是一个激活函数，二分类问题中，
    # sigmoid可以班实数映射到【0,1】，作为概率值，
    # 多分类用softmax函数
).to(device)
# print(dnet)
dnet.to("cpu")
summary(dnet, (1, 1, num_kc), batch_size=1, device="cpu")
dnet.to(device)
# 初始化权重
def weights_init(m):
    if type(m) in [nn.ConvTranspose2d, nn.Conv2d]:
        init.xavier_normal_(m.weight)
    elif type(m) == nn.BatchNorm2d:
        init.normal_(m.weight, 1.0, 0.02)
        init.constant_(m.bias, 0)


gnet.apply(weights_init)
dnet.apply(weights_init)

### GAN 数据加载，构建数据集

In [ ]:
# 加载 openGAN 所需数据，这里只有训练、验证两部分，没有测试集，因为本次数据采集划分时只划出两部分，不影响最终结果
transform = transforms.Compose(
    [
        transforms.ToTensor(),  # 将图片转换为Tensor,归一化至[0,1]
    ]
)
GAN_dataarray = np.load("../../data/nina_db1/t03.2/s1/Fopg_s1_diff_ratio_vec_20220712.npy", allow_pickle=True)
GANdataset = GAN_dataarray.item()
print(type(GANdataset))

ktr_str_X = "vec_ktr_X_"+str(num_kc)+"c"
ktr_str_Y = "ktr_Y_"+str(num_kc)+"c"
val_str_X = "vec_val_X_"+str(num_kc)+"c"
val_str_Y = "vec_val_Y_"+str(num_kc)+"c"
dte_str_X = "vec_dte_X_"+str(num_kc)+"c"
dte_str_Y = "dte_Y_"+str(num_kc)+"c"

traindata = GANdataset[ktr_str_X]
trainlabel = GANdataset[ktr_str_Y]
# 本方法中的 test 实际上是个大号的完整版openset,所以标签是01
testdata = GANdataset[dte_str_X]
testlabel = GANdataset[dte_str_Y]
testlabel_for_auc = GANdataset[dte_str_Y]
testlabel_for_auc = testlabel_for_auc.ravel()

valdata = GANdataset[val_str_X]
vallabel = GANdataset[val_str_Y]
vallabel_for_auc = GANdataset[val_str_Y]
vallabel_for_auc = vallabel_for_auc.ravel()

# 注意这里 ktr，kte 的标签需要降维，其他的在数据制作阶段已经降过了
# trainlabel = trainlabel[:, 0]
# testlabel = testlabel[:,0]
# vallabel = vallabel[:, 0]
# trainunknownc_label = trainunknownc_label[:,0]
# print(type(trainlabel))
train_set = EMGDataset_1D(traindata, trainlabel)
test_set = EMGDataset_1D(testdata, testlabel)
val_set = EMGDataset_1D(valdata, vallabel)
# train_unknown = EMGDataset(trainunknown_data,trainunknownc_label)
# train_loader = torch.utils.data.DataLoader(train_set, batch_size=1, shuffle=True, pin_memory=True,
#                                             num_workers=3)

sample = next(iter(train_set))
print(sample)
sample = next(iter(test_set))
print(sample)
sample = next(iter(val_set))
print(sample)

### 训练 GAN

In [ ]:
# GAN 训练,120
train_loader = torch.utils.data.DataLoader(train_set, batch_size=128)
# 损失
criterion = nn.BCEWithLogitsLoss().to(device)
# criterion = F.cross_entropy()
# 优化器
goptimizer = torch.optim.Adam(gnet.parameters(), lr=0.0008 * 1.5, betas=(0.5, 0.999))
doptimizer = torch.optim.Adam(dnet.parameters(), lr=0.4 * 1, betas=(0.5, 0.999))
g_lr_schedule = torch.optim.lr_scheduler.StepLR(
    goptimizer, 1000, gamma=0.95, last_epoch=-1
)
d_lr_schedule = torch.optim.lr_scheduler.StepLR(
    doptimizer, 1000, gamma=0.95, last_epoch=-1
)
# 用于测试的固定噪声,用来查看相同的潜在张量在训练过程中生成图片的变换
# batch_size = 512
# fixed_noises = torch.randn(batch_size, latent_size, 1, 1).to(device)
# 初始化 visdom
viz.close()
viz = viz_init()
viz = Visdom(
    env=viz_acnt.evns, log_to_filename=vislogDir + "vislog_" + get_current_time()
)
vizx = 0
# viz.text(
#     "MONITOR: Show train process~~",
#     win="Monitor",
#     opts={
#         "title": "ProcessMonitor",
#     },
# )
# AUC值
auc_current = 0
auc_max = 0
auc_current_te = 0
auc_max_te = 0
# 训练过程
epoch_num = 2500
for epoch in range(epoch_num):
    # viz.text(
    #     "ep" + str(epoch + 1) + " start",
    #     win="Monitor",
    #     opts={
    #         "title": "ProcessMonitor",
    #     },
    # )
    dnet.to(device)
    for batch in train_loader:
        # 载入本批次数据
        # real_images, _ = data
        # viz.text('load data',win='Monitor',append=True,opts = {'title':'ProcessMonitor',},)
        rimages, rlabels = batch
        rimages = rimages.to(device)
        rlabels = rlabels.to(device)
        rimages = rimages.to(torch.float32)
        rlabels = rlabels.long()
        batch_size = rimages.size(0)
        # batch_size = 1344
        # viz.text('training',win='Monitor',append=True,opts = {'title':'ProcessMonitor',},)
        # 训练鉴别网络
        dnet.train()
        gnet.train()
        labels = torch.ones(batch_size)  # 真实数据对应标签为1
        labels = labels.to(device)
        preds = dnet(rimages)  # 对真实数据进行判别
        preds = preds.reshape(-1)

        dloss_real = criterion(preds, labels)  # 真实数据的鉴别器损失
        dmean_real = preds.sigmoid().mean()
        # 计算鉴别器将多少比例的真数据判定为真,仅用于输出显示

        noises = torch.randn(batch_size, latent_size, 1, 1)  # 潜在噪声
        noises = noises.to(device)
        fake_images = gnet(noises)  # 生成假数据
        labels = torch.zeros(batch_size)  # 假数据对应标签为0
        labels = labels.to(device)
        fake = fake_images.detach()
        # 使得梯度的计算不回溯到生成网络,可用于加快训练速度.删去此步结果不变
        preds = dnet(fake)  # 对假数据进行鉴别
        preds = preds.view(-1)
        dloss_fake = criterion(preds, labels)  # 假数据的鉴别器损失
        dmean_fake = preds.sigmoid().mean()
        # 计算鉴别器将多少比例的假数据判定为真,仅用于输出显示

        dloss = dloss_real + dloss_fake  # 总的鉴别器损失
        dnet.zero_grad()
        dloss.backward()
        doptimizer.step()
        # now_dloss += dloss.item()
        # 训练生成网络
        # viz.text(' Generator training',win='Monitor',append=True,opts = {'title':'ProcessMonitor',},)
        labels = torch.ones(batch_size)
        labels = labels.to(device)
        # 生成网络希望所有生成的数据都被认为是真数据
        preds = dnet(fake_images)  # 把假数据通过鉴别网络
        preds = preds.view(-1)
        gloss = criterion(preds, labels)  # 真数据看到的损失
        gmean_fake = preds.sigmoid().mean()
        # 计算鉴别器将多少比例的假数据判定为真,仅用于输出显示
        gnet.zero_grad()
        gloss.backward()
        goptimizer.step()

        # viz.text('train Generator',win='Monitor',append=True,opts = {'title':'ProcessMonitor',},)
        print(
            "[{}/{}]".format(epoch + 1, epoch_num)
            + "D loss:{:g} G loss:{:g}".format(dloss, gloss)
            + "真判真:{:g} 假判真:{:g}/{:g}".format(dmean_real, dmean_fake, gmean_fake)
        )

    # 可视化，每 epoch 更新
    viz.line(
        [float(gloss)],
        [epoch],
        win="loss_perEpoch",
        name="G_loss",
        update="append",
        opts=dict(title="loss_perEpoch", xlabel="epoch", ylabel="loss"),
    )
    viz.line(
        [float(dloss)], [epoch], win="loss_perEpoch", name="D_loss", update="append"
    )
    viz.line(
        [float(goptimizer.state_dict()["param_groups"][0]["lr"])],
        [epoch],
        win="lr_perEpoch",
        name="G_loss",
        update="append",
        opts=dict(title="lr_perEpoch", xlabel="epoch", ylabel="lr"),
    )
    viz.line(
        [float(doptimizer.state_dict()["param_groups"][0]["lr"])],
        [epoch],
        win="lr_perEpoch",
        name="D_loss",
        update="append",
    )
    # viz.line(
    #     [float(dmean_real)],
    #     [epoch],
    #     win="D real_perEpoch",
    #     update="append",
    #     opts=dict(title="D real真判真_perEpoch", xlabel="epoch"),
    # )
    viz.line(
        [float(gmean_fake)],
        [epoch],
        win="D fake_perEpoch",
        update="append",
        opts=dict(title="D fake假判真_perEpoch", xlabel="epoch"),
    )
    # 更新学习率
    g_lr_schedule.step()
    d_lr_schedule.step()
    viz.text(
        "updating weights",
        win="Monitor",
        append=True,
        opts={
            "title": "ProcessMonitor",
        },
    )
    # 定期保存
    if (epoch + 1) % 500 == 0:
        timeForSave = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
        checkpointPath_g = (
            ckpDir + "g_ep_" + str(epoch + 1) + "_" + timeForSave + ".pth"
        )
        checkpointPath_d = (
            ckpDir + "d_ep_" + str(epoch + 1) + "_" + timeForSave + ".pth"
        )
        d_state = {
            "model": dnet.state_dict(),
            "optimizer": doptimizer.state_dict(),
            "epoch": epoch,
        }
        torch.save(d_state, checkpointPath_d)
        g_state = {
            "model": gnet.state_dict(),
            "optimizer": goptimizer.state_dict(),
            "epoch": epoch,
        }
        torch.save(g_state, checkpointPath_g)
        viz.text(
            "epoch " + str(epoch + 1) + " model saved",
            win="Monitor",
            append=True,
            opts={
                "title": "ProcessMonitor",
            },
        )

    #     计算验证集上的 AUC 值
    viz.text(
        "calculate auc start",
        win="Monitor",
        opts={
            "title": "ProcessMonitor",
        },
    )
    dnet.eval()
    dnet.to("cpu")
    emg_vec = emgdata_to_net_preds_sigmoid(data_set=val_set, net_vector=dnet)
    emg_vec = emg_vec.ravel()
    auc_current = metrics.roc_auc_score(vallabel_for_auc, emg_vec, average="micro")
    # ceshiji
    emg_vec_te = emgdata_to_net_preds_sigmoid(data_set=test_set, net_vector=dnet)
    emg_vec_te = emg_vec_te.ravel()
    auc_current_te = metrics.roc_auc_score(testlabel_for_auc, emg_vec_te, average="micro")

    viz.line(
        [auc_current],
        [epoch],
        win="AUC",
        update="append",
        name="valiate",
        opts=dict(title="AUC", xlabel="epoch", ylabel="AUC"),
    )
        
    #     保存策略：整个训练过程中最大的AUC及所有AUC值超过0.8的对应模型，由于最大值源于实时比较，刚开始会有一部分低AUC模型被保存，
    #     无需在意，过后删去即可
    if auc_current > auc_max:
        auc_max = auc_current
        viz.text(
            "max auc: " + str(auc_max) + ", in ep " + str(epoch + 1),
            win="message",
            append=False,
            opts={
                "title": "Max AUC",
            },
        )
                # ceshiji
        emg_vec_te = emgdata_to_net_preds_sigmoid(data_set=test_set, net_vector=dnet)
        emg_vec_te = emg_vec_te.ravel()
        auc_current_te = metrics.roc_auc_score(testlabel_for_auc, emg_vec_te, average="micro")
        auc_max_te = auc_current_te
        viz.text(
            "max auc_te: " + str(auc_max_te) + ", in ep " + str(epoch + 1),
            win="message_te",
            append=False,
            opts={
                "title": "Max AUC_te",
            },
        )
        max_auc_state = {
            "model": dnet.state_dict(),
            "optimizer": doptimizer.state_dict(),
            "epoch": epoch,
        }
        checkpointPath_auc = (
            ckpDir_auc
            + "auc_"
            + str(num_kc) +"kc"+
            + str(int(auc_max * 1000))
            + "d_ep_"
            + str(epoch + 1)
            + "_"
            + timeForSave
            + ".pth"
        )
        torch.save(max_auc_state, checkpointPath_auc)
        viz.text(
            "max auc: "
            + str(auc_max)
            + ", in ep "
            + str(epoch + 1)
            + " saved at "
            + checkpointPath_auc,
            win="message",
            append=True,
            opts={
                "title": "Max AUC",
            },
        )
    elif auc_current >= 0.8:
        checkpointPath_auc = (
            ckpDir_auc
            + "auc_"
            + str(num_kc) +"kc"+
            + str(int(auc_max * 1000))
            + "d_ep_"
            + str(epoch + 1)
            + "_"
            + timeForSave
            + ".pth"
        )
        torch.save(max_auc_state, checkpointPath_auc)
        viz.text(
            "auc: "
            + str(auc_current)
            + ", in ep "
            + str(epoch + 1)
            + " saved at "
            + checkpointPath_auc,
            win="AUC_Monitor",
            append=True,
            opts={
                "title": "Max AUC",
            },
        )


# 完整训练结束后，模型最终保存；意义不大，但以备不时之需
checkpointPath_model = (
    model_Dir
    + "g_final_"
    + "acc"
    + str(int(total_test_acc * 10000))
    + "_"
    + timeForSave
    + ".pth"
)
torch.save(gnet.state_dict(), checkpointPath_model)
checkpointPath_model = (
    model_Dir
    + "d_final_"
    + "acc"
    + str(int(total_test_acc * 10000))
    + "_"
    + timeForSave
    + ".pth"
)
torch.save(dnet.state_dict(), checkpointPath_model)

## 验证结果
绘制 ROC 曲线

### 原始手势测试集，经过 CNN 后正确率